<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/0511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 10.6 求近义词和类比词

In [6]:
!pip install mxnet-cu100 d2lzh

     |████████████████████████████████| 540.1MB 30kB/s 
Processing /root/.cache/pip/wheels/bb/4a/3e/81075d0b470000f4b5769c936f64b22be31c6bcfa81fd050d6/d2lzh-0.8.11-cp36-none-any.whl


In [3]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.0.130


In [8]:
from mxnet import nd 
from mxnet.contrib import text 

text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

In [9]:
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


In [10]:
glove_6b50d = text.embedding.create('glove', pretrained_file_name='glove.6B.50d.txt')

In [11]:
len(glove_6b50d)

400001

In [12]:
glove_6b50d.token_to_idx['beautiful']

3367

In [13]:
glove_6b50d.idx_to_token[3367]

'beautiful'

In [0]:
def knn(W, x, k):
    cos = nd.dot(W, x.reshape((-1,))) / ((nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [0]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed.get_vecs_by_tokens([query_token]), k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

In [17]:
get_similar_tokens('chip', 9, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics
cosine sim=0.731: semiconductor
cosine sim=0.716: maker
cosine sim=0.708: computer
cosine sim=0.707: microprocessor
cosine sim=0.703: makers
cosine sim=0.701: micro


In [18]:
get_similar_tokens('baby', 9, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl
cosine sim=0.778: newborn
cosine sim=0.765: pregnant
cosine sim=0.762: mom
cosine sim=0.759: child
cosine sim=0.756: toddler
cosine sim=0.750: mother


In [19]:
get_similar_tokens('beautiful', 9, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful
cosine sim=0.825: charming
cosine sim=0.801: beauty
cosine sim=0.774: elegant
cosine sim=0.758: looks
cosine sim=0.736: love
cosine sim=0.735: graceful


In [0]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]]

In [21]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [22]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [23]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [24]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

## 10.7 文本情感分类：使用循环神经网络

In [0]:
import collections 
import d2lzh as d2l 
from mxnet import gluon, init, nd 
from mxnet.contrib import text 
from mxnet.gluon import data as gdata, loss as gloss, nn, rnn, utils as gutils 
import os 
import random 
import tarfile 

In [0]:
!mkdir -p ../data/aclImdb

In [48]:
def download_imdb(data_dir='../data'):
    url = ('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
    sha1 = '01ada507287d82875905620988597833ad4e0903'
    fname = gutils.download(url, data_dir, sha1_hash=sha1)
    with tarfile.open(fname, 'r') as f:
        f.extractall(data_dir)

download_imdb()

In [0]:
def read_imdb(folder='train'):
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join('../data/aclImdb/', folder, label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data 

train_data, test_data = read_imdb('train'), read_imdb('test')

In [0]:
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    return [tokenizer(review) for review, _ in data]

In [51]:
def get_vocab_imdb(data):
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return text.vocab.Vocabulary(counter, min_freq=5)

vocab = get_vocab_imdb(train_data)
'# words in vocab:', len(vocab)

('# words in vocab:', 46151)

In [0]:
def preprocess_imdb(data, vocab):
    max_l = 500 

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))
    
    tokenized_data = get_tokenized_imdb(data)
    features = nd.array([pad(vocab.to_indices(x)) for x in tokenized_data])
    labels = nd.array([score for _, score in data])
    return features, labels

In [0]:
batch_size = 64 
train_set = gdata.ArrayDataset(*preprocess_imdb(train_data, vocab))
test_set = gdata.ArrayDataset(*preprocess_imdb(test_data, vocab))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

In [55]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break 
'#batches:', len(train_iter)

X (64, 500) y (64,)


('#batches:', 391)

In [0]:
class BiRNN(nn.Block):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(len(vocab), embed_size)
        self.encoder = rnn.LSTM(num_hiddens, num_layers=num_layers, bidirectional=True, input_size=embed_size)
        self.decoder = nn.Dense(2)

    def forward(self, inputs):
        embeddings = self.embedding(inputs.T)
        outputs = self.encoder(embeddings)
        encoding = nd.concat(outputs[0], outputs[-1])
        outs = self.decoder(encoding)
        return outs 

In [0]:
embed_size, num_hiddens, num_layers, ctx = 100, 100, 2, d2l.try_all_gpus()
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)
net.initialize(init.Xavier(), ctx=ctx)

In [0]:
glove_embedding = text.embedding.create('glove', pretrained_file_name='glove.6B.100d.txt', vocabulary=vocab)

In [0]:
net.embedding.weight.set_data(glove_embedding.idx_to_vec)
net.embedding.collect_params().setattr('grad_req', 'null')

In [61]:
lr, num_epochs = 0.01, 5 
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gloss.SoftmaxCrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs)

training on [gpu(0)]
epoch 1, loss 0.5538, train acc 0.704, test acc 0.811, time 45.7 sec
epoch 2, loss 0.4081, train acc 0.822, test acc 0.835, time 45.5 sec
epoch 3, loss 0.3668, train acc 0.844, test acc 0.812, time 45.1 sec
epoch 4, loss 0.3333, train acc 0.858, test acc 0.846, time 45.3 sec
epoch 5, loss 0.2922, train acc 0.878, test acc 0.847, time 45.3 sec


In [0]:
def predict_sentiment(net, vocab, sentence):
    sentence = nd.array(vocab.to_indices(sentence), ctx=d2l.try_gpu())
    label = nd.argmax(net(sentence.reshape((1, -1))), axis=1)
    return 'positive' if label.asscalar() == 1 else 'negative'

In [63]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'

In [64]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])

'negative'

## 10.8 文本情感分类：使用卷积神经网络 ( textCNN )

In [0]:
import d2lzh as d2l 
from mxnet import gluon, init, nd 
from mxnet.contrib import text 
from mxnet.gluon import data as gdata, loss as gloss, nn 

In [0]:
def corr1d(X, K):
    w = K.shape[0]
    Y = nd.zeros((X.shape[0] - w + 1))
    for i in range(Y.shape[0]):
        Y[i] = (X[i: i + w] * K).sum()
    return Y 

In [68]:
X, K = nd.array([0, 1, 2, 3, 4, 5, 6]), nd.array([1, 2])
corr1d(X, K)


[ 2.  5.  8. 11. 14. 17.]
<NDArray 6 @cpu(0)>

In [69]:
def corr1d_multi_in(X, K):
    return nd.add_n(*[corr1d(x, k) for x, k in zip(X, K)])

X = nd.array([[0, 1, 2, 3, 4, 5, 6], 
        [1, 2, 3, 4, 5, 6, 7], 
        [2, 3, 4, 5, 6, 7, 8]])
K = nd.array([[1, 2], [3, 4], [-1, -3]])
corr1d_multi_in(X, K)


[ 2.  8. 14. 20. 26. 32.]
<NDArray 6 @cpu(0)>

In [0]:
batch_size = 64 
d2l.download_imdb()
train_data, test_data = d2l.read_imdb('train'), d2l.read_imdb('test')
vocab = d2l.get_vocab_imdb(train_data)
train_iter = gdata.DataLoader(gdata.ArrayDataset(*d2l.preprocess_imdb(train_data, vocab)), batch_size, shuffle=True)
test_iter = gdata.DataLoader(gdata.ArrayDataset(*d2l.preprocess_imdb(test_data, vocab)), batch_size)

In [0]:
class TextCNN(nn.Block):
    def __init__(self, vocab, embed_size, kernel_size, num_channels, **kwargs):
        super(TextCNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(len(vocab), embed_size)
        self.constant_embedding = nn.Embedding(len(vocab), embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Dense(2)
        self.pool = nn.GlobalMaxPool1D()
        self.convs = nn.Sequential()
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.add(nn.Conv1D(c, k, activation='relu'))

    def forward(self, inputs):
        embeddings = nd.concat(self.embedding(inputs), self.constant_embedding(inputs), dim=2)
        embeddings = embeddings.transpose((0, 2, 1))
        encoding = nd.concat(*[nd.flatten(self.pool(conv(embeddings))) for conv in self.convs], dim=1)
        outputs = self.decoder(self.dropout(encoding))
        return outputs 

In [0]:
embed_size, kernel_sizes, nums_channels = 100, [3, 4, 5], [100, 100, 100]
ctx = d2l.try_all_gpus()
net = TextCNN(vocab, embed_size, kernel_sizes, nums_channels)
net.initialize(init.Xavier(), ctx=ctx)

In [0]:
glove_embedding = text.embedding.create('glove', pretrained_file_name='glove.6B.100d.txt', vocabulary=vocab) 
net.embedding.weight.set_data(glove_embedding.idx_to_vec)
net.constant_embedding.weight.set_data(glove_embedding.idx_to_vec)
net.constant_embedding.collect_params().setattr('grad_req', 'null')

In [74]:
lr, num_epochs = 0.001, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gloss.SoftmaxCrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs)

training on [gpu(0)]
epoch 1, loss 0.6080, train acc 0.716, test acc 0.832, time 19.3 sec
epoch 2, loss 0.3614, train acc 0.839, test acc 0.854, time 18.9 sec
epoch 3, loss 0.2665, train acc 0.891, test acc 0.821, time 18.9 sec
epoch 4, loss 0.1764, train acc 0.933, test acc 0.862, time 18.9 sec
epoch 5, loss 0.1024, train acc 0.965, test acc 0.865, time 18.9 sec


In [75]:
d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'

In [76]:
d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])

'negative'